In [ ]:
from collections import deque
import copy

In [ ]:
ROWS = range(9)
COLS = range(9)

# Variables: all (row, col) pairs
VARS = [(r, c) for r in ROWS for c in COLS]


def subgrid_index(r, c):
    return r // 3, c // 3


NEIGHBORS = {v: set() for v in VARS}
for r in ROWS:
    for c in COLS:
        v = (r, c)

        # Same row
        for cc in COLS:
            if cc != c:
                NEIGHBORS[v].add((r, cc))

        # Same column
        for rr in ROWS:
            if rr != r:
                NEIGHBORS[v].add((rr, c))

        # Same 3x3 block
        br, bc = subgrid_index(r, c)
        for rr in range(br * 3, br * 3 + 3):
            for cc in range(bc * 3, bc * 3 + 3):
                if (rr, cc) != v:
                    NEIGHBORS[v].add((rr, cc))

In [ ]:
def init_domains(board):
    domains = {v: set(range(1, 10)) for v in VARS}

    # Set domains for given cells
    for r in ROWS:
        for c in COLS:
            val = board[r][c]
            if val != 0:
                domains[(r, c)] = {val}

    # Prune neighbors using the givens
    for r in ROWS:
        for c in COLS:
            val = board[r][c]
            if val != 0:
                for n in NEIGHBORS[(r, c)]:
                    if val in domains[n] and len(domains[n]) > 1:
                        domains[n].discard(val)

    return domains

In [ ]:
def is_consistent(value, var, assignment):
    for n in NEIGHBORS[var]:
        if n in assignment and assignment[n] == value:
            return False
    return True

In [ ]:
# Heuristics: MRV + Degree

def select_unassigned_variable(domains, assignment):
    unassigned = [v for v in VARS if v not in assignment]

    # MRV: minimum domain size
    min_size = min(len(domains[v]) for v in unassigned)
    candidates = [v for v in unassigned if len(domains[v]) == min_size]

    if len(candidates) == 1:
        return candidates[0]

    # Degree heuristic: most constraints on other unassigned vars
    best = None
    best_degree = -1
    for v in candidates:
        degree = sum(1 for n in NEIGHBORS[v] if n not in assignment)
        if degree > best_degree:
            best_degree = degree
            best = v
    return best

In [ ]:
# Heuristic: LCV (Least Constraining Value)

def order_domain_values(var, domains):
    scores = []
    for val in domains[var]:
        # Count how many neighbor-domain values would be removed
        removed_count = 0
        for n in NEIGHBORS[var]:
            if val in domains[n]:
                removed_count += 1
        scores.append((removed_count, val))

    # Least constraining first (smallest removed_count)
    scores.sort()
    return [val for (_, val) in scores]

In [ ]:
# Forward checking

def forward_check(var, value, domains):
    new_domains = copy.deepcopy(domains)
    new_domains[var] = {value}

    for n in NEIGHBORS[var]:
        if value in new_domains[n]:
            # If neighbor already forced to this value, we must not remove its only domain value
            if len(new_domains[n]) == 1:
                return False, None
            new_domains[n].remove(value)
            if len(new_domains[n]) == 0:
                return False, None

    return True, new_domains

In [ ]:
# AC-3 (Arc Consistency)

def ac3(domains):
    queue = deque()
    for Xi in VARS:
        for Xj in NEIGHBORS[Xi]:
            queue.append((Xi, Xj))

    dom = {v: set(vals) for v, vals in domains.items()}

    def revise(Xi, Xj):
        removed = False
        to_remove = set()

        # For each x in domain(Xi), check if there is a y in domain(Xj) such that x != y.
        for x in dom[Xi]:
            if not any(x != y for y in dom[Xj]):
                # x has no supporting value in Xj
                to_remove.add(x)

        if to_remove:
            dom[Xi] -= to_remove
            removed = True

        return removed

    while queue:
        Xi, Xj = queue.popleft()
        if revise(Xi, Xj):
            if len(dom[Xi]) == 0:
                return False, domains  # failure
            for Xk in NEIGHBORS[Xi]:
                if Xk != Xj:
                    queue.append((Xk, Xi))

    return True, dom

In [ ]:
# Backtracking search

def backtrack(assignment, domains, use_forward_check=True, use_ac3_in_search=False):
    # All variables assigned => solved
    if len(assignment) == 81:
        return assignment, domains

    # Select variable with MRV + Degree
    var = select_unassigned_variable(domains, assignment)

    # Order values with LCV
    for value in order_domain_values(var, domains):

        if is_consistent(value, var, assignment):
            # Try assign
            assignment[var] = value

            # Apply forward checking if enabled
            if use_forward_check:
                ok, new_domains = forward_check(var, value, domains)
                if not ok:
                    del assignment[var]
                    continue
                working_domains = new_domains
            else:
                working_domains = domains

            # Optionally run AC-3 after each assignment
            if use_ac3_in_search:
                ok_ac3, ac_domains = ac3(working_domains)
                if not ok_ac3:
                    del assignment[var]
                    continue
                working_domains_2 = ac_domains
            else:
                working_domains_2 = working_domains

            # Recurse
            result, final_domains = backtrack(
                assignment,
                working_domains_2,
                use_forward_check,
                use_ac3_in_search,
            )
            if result is not None:
                return result, final_domains

            # Backtrack
            del assignment[var]

    return None, domains

In [ ]:
# Top-level Sudoku solver

def solve_sudoku(board, use_forward_check=True, use_ac3_pre=True, use_ac3_in_search=False):
    domains = init_domains(board)

    # Initial assignment from given digits
    assignment = {}
    for r in ROWS:
        for c in COLS:
            if board[r][c] != 0:
                assignment[(r, c)] = board[r][c]

    # Run AC-3 once before search (typical)
    if use_ac3_pre:
        ok, domains = ac3(domains)
        if not ok:
            return None  # inconsistent puzzle

    # Backtracking search
    result, final_domains = backtrack(
        assignment,
        domains,
        use_forward_check=use_forward_check,
        use_ac3_in_search=use_ac3_in_search,
    )

    if result is None:
        return None

    # Build solved board
    solved = [[0] * 9 for _ in range(9)]
    for (r, c), v in result.items():
        solved[r][c] = v

    return solved

In [ ]:
# Helper: pretty print

def print_board(board):
    for r in range(9):
        row = ""
        for c in range(9):
            val = board[r][c]
            row += (str(val) if val != 0 else "_") + " "
            if c % 3 == 2 and c != 8:
                row += "| "
        print(row)
        if r % 3 == 2 and r != 8:
            print("-" * 21)

In [ ]:
# Example usage with your given puzzle

if __name__ == "__main__":
    puzzle = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9],
    ]

    print("Initial puzzle:")
    print_board(puzzle)

    solution = solve_sudoku(
        puzzle,
        use_forward_check=True,
        use_ac3_pre=True,
        use_ac3_in_search=True,  # you can set False if you want simpler FC+MRV+LCV
    )

    print("\nSolved puzzle:")
    print_board(solution)


Initial puzzle:
5 3 _ | _ 7 _ | _ _ _ 
6 _ _ | 1 9 5 | _ _ _ 
_ 9 8 | _ _ _ | _ 6 _ 
---------------------
8 _ _ | _ 6 _ | _ _ 3 
4 _ _ | 8 _ 3 | _ _ 1 
7 _ _ | _ 2 _ | _ _ 6 
---------------------
_ 6 _ | _ _ _ | 2 8 _ 
_ _ _ | 4 1 9 | _ _ 5 
_ _ _ | _ 8 _ | _ 7 9 

Solved puzzle:
5 3 4 | 6 7 8 | 9 1 2 
6 7 2 | 1 9 5 | 3 4 8 
1 9 8 | 3 4 2 | 5 6 7 
---------------------
8 5 9 | 7 6 1 | 4 2 3 
4 2 6 | 8 5 3 | 7 9 1 
7 1 3 | 9 2 4 | 8 5 6 
---------------------
9 6 1 | 5 3 7 | 2 8 4 
2 8 7 | 4 1 9 | 6 3 5 
3 4 5 | 2 8 6 | 1 7 9 
